#### **Project Title:**  ***" Facial Emotion Recognition"***
##### **Dataset:** *"FER 2013 Dataset"*
##### **Step 01:** Loading Important Libraries

In [ ]:
#Importing the required libraries.
#To download the FER 2013 dataset.
import kagglehub
#For data preprocessing and augmentation.
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#For transfer learning with a pre-trained CNN model.
from tensorflow.keras.applications import VGG16
#For building the custom classification model.
from tensorflow.keras.models import Model
#For constructing the classifier.
from tensorflow.keras.layers import Dense, Flatten, Dropout
#For training the model.
from tensorflow.keras.optimizers import Adam
#For alternative transfer learning with a pre-trained CNN model.
from tensorflow.keras.applications import ResNet50
#For evaluating the model's performance.
from sklearn.metrics import classification_report, accuracy_score, f1_score
#For numerical computations.
import numpy as np

##### **Step 02:** Loading the Dataset

In [1]:
#Downloading the latest version of the FER 2013 dataset from Kaggle using KaggleHub.
path = kagglehub.dataset_download("msambare/fer2013")
#Printing the path where the dataset files have been saved after downloading and extracting.
print("Path to dataset files:", path)

100%|██████████| 60.3M/60.3M [00:00<00:00, 90.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/msambare/fer2013/versions/1


##### **Step 03:** Data Preprocessing(Resizing, Normalizing, Augmenting Data)

In [9]:
#Defining the paths to the training and testing directories of the FER 2013 dataset.
train_dir = "/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/train"
test_dir = "/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/test"
#Creating an ImageDataGenerator for preprocessing and augmenting the dataset.
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,       #Normalize pixel values to the range [0, 1].
    rotation_range=20,         #Randomly rotate images by up to 20 degrees for augmentation.
    width_shift_range=0.2,     #Randomly shift images horizontally by up to 20% of the width.
    height_shift_range=0.2,    #Randomly shift images vertically by up to 20% of the height.
    horizontal_flip=True,      #Randomly flip images horizontally.
    validation_split=0.2       #Reserve 20% of the training data for validation.
)
#Loading the training data, applying augmentation.
train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),      #Resize all images to 48x48 pixels.
    color_mode="rgb",          #Convert grayscale images to RGB format (3 channels).
    class_mode="categorical",  #Use one-hot encoding for class labels.
    batch_size=32,             #Process images in batches of 32.
    subset="training"          #Load the training subset.
)
#Loading the validation data, applying the same preprocessing as training data.
val_data = datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),      #Resize all images to 48x48 pixels.
    color_mode="rgb",          #Convert grayscale images to RGB format (3 channels).
    class_mode="categorical",  #Use one-hot encoding for class labels.
    batch_size=32,             #Process images in batches of 32.
    subset="validation"        #Load the validation subset.
)
#Creating a separate ImageDataGenerator for the test data (no augmentation, only normalization).
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)  #Normalize pixel values to [0, 1].
#Loading the test data.
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),      #Resize all images to 48x48 pixels.
    color_mode="rgb",          #Convert grayscale images to RGB format (3 channels).
    class_mode="categorical",  #Use one-hot encoding for class labels.
    batch_size=32              #Process images in batches of 32.
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


##### **Step 04:** Transfer Learning with a Pre-Trained CNN Model (VGG16/ResNet)

In [10]:
#Loading the VGG16 model without the top layers.
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(48, 48, 3))
#Freezing all the layers in the base model to retain pre-trained features.
for layer in base_model.layers:
    layer.trainable = False  #Prevent updates to the weights during training.
#Adding custom layers for emotion classification.
x = base_model.output          #Start with the output of the base model.
x = Flatten()(x)               #Flatten the feature maps into a 1D vector.
x = Dense(256, activation="relu")(x)  #Add a dense (fully connected) layer with 256 units and ReLU activation.
x = Dropout(0.5)(x)            #Apply dropout to reduce overfitting.
output = Dense(7, activation="softmax")(x)  #Add the output layer with 7 units (one for each emotion class) and softmax activation.
#Creating the final model by connecting the base model and custom layers.
model = Model(inputs=base_model.input, outputs=output)
#Compiling the model with Adam optimizer, categorical crossentropy loss and accuracy as the evaluation metric.
model.compile(
    optimizer=Adam(learning_rate=1e-4),  #Use a small learning rate for stable training.
    loss="categorical_crossentropy",     #Suitable for multi-class classification.
    metrics=["accuracy"]                 #Monitor accuracy during training.
)
#Training the model on the training data and validate on the validation data.
history = model.fit(
    train_data,                          #Training data generator.
    validation_data=val_data,            #Validation data generator.
    epochs=10,                           #Number of epochs to train.
    steps_per_epoch=train_data.samples // train_data.batch_size,  #Steps per epoch for training.
    validation_steps=val_data.samples // val_data.batch_size      #Steps per epoch for validation.
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


717/717 ━━━━━━━━━━━━━━━━━━━━ 838s 1s/step - accuracy: 0.2093 - loss: 1.9684 - val_accuracy: 0.3017 - val_loss: 1.7342
Epoch 2/10
  1/717 ━━━━━━━━━━━━━━━━━━━━ 9:15 775ms/step - accuracy: 0.3438 - loss: 1.6800

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


717/717 ━━━━━━━━━━━━━━━━━━━━ 1s 535us/step - accuracy: 0.3438 - loss: 1.6800 - val_accuracy: 0.1538 - val_loss: 1.9002
Epoch 3/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 828s 1s/step - accuracy: 0.2850 - loss: 1.7547 - val_accuracy: 0.3205 - val_loss: 1.7029
Epoch 4/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 1s 471us/step - accuracy: 0.1875 - loss: 1.8783 - val_accuracy: 0.3077 - val_loss: 1.6650
Epoch 5/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 831s 1s/step - accuracy: 0.3148 - loss: 1.7155 - val_accuracy: 0.3303 - val_loss: 1.6887
Epoch 6/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 1s 464us/step - accuracy: 0.2188 - loss: 1.8342 - val_accuracy: 0.0000e+00 - val_loss: 2.0276
Epoch 7/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 826s 1s/step - accuracy: 0.3126 - loss: 1.7051 - val_accuracy: 0.3319 - val_loss: 1.6735
Epoch 8/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 1s 470us/step - accuracy: 0.4688 - loss: 1.4732 - val_accuracy: 0.2308 - val_loss: 2.0098
Epoch 9/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 826s 1s/step - accuracy: 0.3260 - loss: 1.6872 - val_accurac

In [13]:
#Loading the ResNet50 model without the top layers.
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(48, 48, 3))
#Freezing all the layers in the base model to retain pre-trained features.
for layer in base_model.layers:
    layer.trainable = False  #Prevent updates to the weights of the pre-trained layers during training.
#Adding custom layers for emotion classification.
x = base_model.output          #Start with the output of the base model.
x = Flatten()(x)               #Flatten the feature maps into a 1D vector.
x = Dense(256, activation="relu")(x)  #Add a dense (fully connected) layer with 256 units and ReLU activation.
x = Dropout(0.5)(x)            #Apply dropout with a 50% rate to reduce overfitting.
output = Dense(7, activation="softmax")(x)  #Add the output layer with 7 units (one for each emotion class) and softmax activation.
#Creating the final model by combining the base model and custom layers.
model = Model(inputs=base_model.input, outputs=output)
#Compiling the model with Adam optimizer, categorical crossentropy loss and accuracy as the evaluation metric.
model.compile(
    optimizer=Adam(learning_rate=1e-4),  #Use a small learning rate for stable training.
    loss="categorical_crossentropy",     #Suitable for multi-class classification.
    metrics=["accuracy"]                 #Monitor accuracy during training.
)
#Training the model on the training data and validate on the validation data.
history = model.fit(
    train_data,                          #Training data generator.
    validation_data=val_data,            #Validation data generator.
    epochs=10,                           #Number of epochs to train.
    steps_per_epoch=train_data.samples // train_data.batch_size,  #Steps per epoch for training.
    validation_steps=val_data.samples // val_data.batch_size      #Steps per epoch for validation.
)

Epoch 1/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 340s 463ms/step - accuracy: 0.2202 - loss: 1.9098 - val_accuracy: 0.2505 - val_loss: 1.8093
Epoch 2/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 17s 23ms/step - accuracy: 0.3125 - loss: 1.9497 - val_accuracy: 0.2308 - val_loss: 1.7649
Epoch 3/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 363s 462ms/step - accuracy: 0.2342 - loss: 1.8247 - val_accuracy: 0.2533 - val_loss: 1.8037
Epoch 4/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 0s 252us/step - accuracy: 0.1562 - loss: 1.8957 - val_accuracy: 0.2308 - val_loss: 1.7916
Epoch 5/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 381s 461ms/step - accuracy: 0.2394 - loss: 1.8261 - val_accuracy: 0.2512 - val_loss: 1.8011
Epoch 6/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 1s 420us/step - accuracy: 0.2500 - loss: 1.7495 - val_accuracy: 0.3846 - val_loss: 1.7962
Epoch 7/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 382s 462ms/step - accuracy: 0.2446 - loss: 1.8186 - val_accuracy: 0.2519 - val_loss: 1.8006
Epoch 8/10
717/717 ━━━━━━━━━━━━━━━━━━━━ 0s 267us/step - accuracy: 0.1562 - loss: 1

##### **Step 05:** Fine-Tuning the Model and Evaluating Its Performance

In [14]:
#Unfreezing the last few layers of the base model for fine-tuning.
for layer in base_model.layers[-4:]:  #Unfreeze the last 4 layers.
    layer.trainable = True
#Recompiling the model with a smaller learning rate.
model.compile(
    optimizer=Adam(learning_rate=1e-5),  #Smaller learning rate for fine-tuning.
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
#Fine-tuning the model.
history_fine_tune = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,  #Additional epochs for fine-tuning.
    steps_per_epoch=train_data.samples // train_data.batch_size,
    validation_steps=val_data.samples // val_data.batch_size
)

Epoch 1/5
717/717 ━━━━━━━━━━━━━━━━━━━━ 357s 486ms/step - accuracy: 0.2347 - loss: 1.8242 - val_accuracy: 0.2544 - val_loss: 1.7957
Epoch 2/5
717/717 ━━━━━━━━━━━━━━━━━━━━ 1s 454us/step - accuracy: 0.2812 - loss: 1.6948 - val_accuracy: 0.2308 - val_loss: 1.8287
Epoch 3/5
717/717 ━━━━━━━━━━━━━━━━━━━━ 382s 487ms/step - accuracy: 0.2521 - loss: 1.8047 - val_accuracy: 0.2556 - val_loss: 1.7849
Epoch 4/5
717/717 ━━━━━━━━━━━━━━━━━━━━ 0s 249us/step - accuracy: 0.2188 - loss: 1.8573 - val_accuracy: 0.1538 - val_loss: 1.9231
Epoch 5/5
717/717 ━━━━━━━━━━━━━━━━━━━━ 349s 486ms/step - accuracy: 0.2490 - loss: 1.8043 - val_accuracy: 0.2558 - val_loss: 1.7872


In [15]:
#Evaluating the model on validation data.
val_loss, val_accuracy = model.evaluate(val_data)
print("Validation Loss after Fine-Tuning:", val_loss)
print("Validation Accuracy after Fine-Tuning:", val_accuracy)
#Evaluating the model on test data.
test_loss, test_accuracy = model.evaluate(test_data)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

180/180 ━━━━━━━━━━━━━━━━━━━━ 63s 348ms/step - accuracy: 0.2467 - loss: 1.7943
Validation Loss after Fine-Tuning: 1.7866168022155762
Validation Accuracy after Fine-Tuning: 0.25831738114356995
225/225 ━━━━━━━━━━━━━━━━━━━━ 73s 326ms/step - accuracy: 0.2671 - loss: 1.7778
Test Loss: 1.777564287185669
Test Accuracy: 0.26469770073890686


##### **Step 06:** Reporting metrics 

In [16]:
#Getting ground truth labels and predictions.
test_labels = test_data.classes  #True labels.
class_names = list(test_data.class_indices.keys())  #Emotion class names.
#Predicting probabilities.
predictions = model.predict(test_data, steps=test_data.samples // test_data.batch_size + 1)
#Converting predicted probabilities to class labels.
predicted_classes = np.argmax(predictions, axis=1)
#Reporting accuracy.
accuracy = accuracy_score(test_labels, predicted_classes)
print(f"Test Accuracy: {accuracy:.4f}")
#Reporting F1-score for each class.
f1_scores = f1_score(test_labels, predicted_classes, average=None)
for i, score in enumerate(f1_scores):
    print(f"F1-Score for {class_names[i]}: {score:.4f}")
#Generating a detailed classification report.
report = classification_report(test_labels, predicted_classes, target_names=class_names)
print("\nClassification Report:\n")
print(report)

225/225 ━━━━━━━━━━━━━━━━━━━━ 81s 332ms/step
Test Accuracy: 0.2329
F1-Score for angry: 0.0261
F1-Score for disgust: 0.0000
F1-Score for fear: 0.0149
F1-Score for happy: 0.3864
F1-Score for neutral: 0.0492
F1-Score for sad: 0.0109
F1-Score for surprise: 0.0834

Classification Report:

              precision    recall  f1-score   support

       angry       0.12      0.01      0.03       958
     disgust       0.00      0.00      0.00       111
        fear       0.15      0.01      0.01      1024
       happy       0.25      0.87      0.39      1774
     neutral       0.16      0.03      0.05      1233
         sad       0.20      0.01      0.01      1247
    surprise       0.11      0.07      0.08       831

    accuracy                           0.23      7178
   macro avg       0.14      0.14      0.08      7178
weighted avg       0.17      0.23      0.12      7178



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### **Conclusion**

##### This project utilized the **FER 2013 dataset** and transfer learning with **VGG16** to classify emotions. The model achieved a **23.29% test accuracy**, with "happy" being the best-recognized emotion (**F1-score: 0.39**), while other emotions, such as "disgust" and "sad," had near-zero F1-scores.

#### **Key Insights**
1. **Dataset Imbalance**  
   The highly imbalanced dataset significantly impacted model performance, especially for underrepresented emotions like "disgust."
2. **Subtle Features**  
   General pre-trained models like VGG16 struggled to capture the nuanced facial expressions required for accurate emotion recognition.
3. **Result Portrayal**  
   The low accuracy and poor F1-scores highlight that the dataset and current approach are insufficient for reliable emotion recognition.
The results underscore the need for more balanced datasets and specialized models to achieve robust facial emotion recognition. While this project lays a foundation, further enhancements are necessary to improve accuracy and generalization. 
---